In [4]:
import numpy as np
import torch
import pandas as pd
import random
import csv
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# 기본 작업 시작 #
class TreeNode:
    def __init__(self, path, cost):
        self.path = path
        self.cost = cost
        self.children = []

device = torch.device("cpu")

def load_data(filepath): # csv 파일 읽어 도시 저장
    with open(filepath, mode='r', newline='') as file:
        data = [list(map(float, row)) for row in csv.reader(file)]
    tensor_data = torch.tensor(data, dtype=torch.float32).to(device)
    return tensor_data

def plot_path(path, coordinates):
    coordinates = coordinates.cpu().numpy()
    path = np.array(path + [path[0]])
    plt.figure(figsize=(10, 10))
    plt.plot(coordinates[path, 0], coordinates[path, 1], 'o-')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.title('Optimized Path')
    plt.show()

# 기본 작업 끝 #

# 0 세대 관련 작업 시작 #
def MST(cities): # 괜찮은 population(0세대)를 만들기 위한 MST 알고리즘
    path = [0]  # 시작 노드를 0번 도시로 설정
    visited = set([0])
    current_city = 0
    while len(path) < len(cities):  # 모든 도시를 방문할 때까지
        next_city = None
        min_dist = np.inf
        for neighbor in range(len(cities)):
            if neighbor not in visited:
                dist = np.linalg.norm(cities[current_city] - cities[neighbor])
                if dist < min_dist:
                    min_dist = dist
                    next_city = neighbor
        path.append(next_city)
        visited.add(next_city)
        current_city = next_city

    # 마지막 경로를 0으로 설정
    path.append(0)
    # 경로의 길이 계산
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += np.linalg.norm(cities[path[i]] - cities[path[i+1]])
    total_distance += np.linalg.norm(cities[path[-1]] - cities[path[0]])  # 마지막 도시에서 출발 도시로의 거리 추가
    return path, len(visited), total_distance

def nearest_neighbor(cities): # 괜찮은 population(0세대)를 만들기 위한 nn 알고리즘
    num_cities = len(cities)
    unvisited_cities = set(range(1, num_cities))
    current_city = 0  # 시작 도시는 첫 번째 도시
    tour = [current_city]  # 경로의 시작은 시작 도시
    while unvisited_cities:
        nearest_city = min(unvisited_cities, key=lambda city: distance(cities[current_city], cities[city])) # 해당 노드에서 가장 가까운 도시 선택
        tour.append(nearest_city)
        unvisited_cities.remove(nearest_city)
        current_city = nearest_city
    tour.append(0)  # 출발 도시로 돌아가는 경로 추가
    return tour

def farthest_neighbor(cities): # Tree의 root노드를 최악에 가까운 경로로 선택하기 위해 nn 알고리즘의 반대
    num_cities = len(cities)
    unvisited_cities = set(range(1, num_cities))
    current_city = 0  # 시작 도시는 첫 번째 도시
    tour = [current_city]  # 경로의 시작은 시작 도시
    while unvisited_cities:
        farthest_city = max(unvisited_cities, key=lambda city: distance(cities[current_city], cities[city])) # 해당 노드에서 가장 먼 도시 선택
        tour.append(farthest_city)
        unvisited_cities.remove(farthest_city)
        current_city = farthest_city
    tour.append(0)  # 출발 도시로 돌아가는 경로 추가
    return tour

def expand_root_node(parent_node,base_node,how_many,coordinates,mutation_rate): # 0세대를 root 노드 아래에 생성
    for _ in range(how_many):
        new_path=mutate(base_node.path,mutation_rate) # base_node로 들어온 0세대 노드를 mutation하여 0세대 노드 추가 생성
        new_cost=calculate_cost(new_path,coordinates)
        new_node=TreeNode(new_path,new_cost)
        parent_node.children.append(new_node)

# 0 세대 관력 작업 끝 #

# 기본 트리 작업 시작 #
def distance(city1, city2): # 도시간 유클리드 거리 계산
    return np.linalg.norm(city1 - city2)

def calculate_cost(path, coordinates): # 하나의 경로의 비용 계산
    idx = torch.tensor(path + [path[0]], device=device)
    return torch.sum(torch.norm(coordinates[idx[:-1]] - coordinates[idx[1:]], dim=1)).item()

def stopping_condition(current_iteration, max_iterations):
    return current_iteration >= max_iterations

def expand_tree(node, num_expansions, coordinates, mutation_rate): # 트리 확장
    for _ in range(num_expansions):
        new_path = mutate(node.path, mutation_rate)
        new_cost = calculate_cost(new_path, coordinates)
        if new_cost>node.cost: # 자식이 부모보다 않 좋을 경우 트리 확장 X
            continue
        new_node = TreeNode(new_path, new_cost)
        node.children.append(new_node)

def find_best_leaf_node(node):
    if not node.children:
        return node
    else:
        leaf_nodes = []
        find_all_leaf_nodes(node, leaf_nodes)
        return min(leaf_nodes, key=lambda x: x.cost)

def find_all_leaf_nodes(node, leaf_nodes):
    if not node.children:
        leaf_nodes.append(node)
    else:
        for child in node.children:
            find_all_leaf_nodes(child, leaf_nodes)

# 기본 트리 작업 끝 #

# 유전 알고리즘 시작 #
def mutate(path, mutation_rate): # 유전 알고리즘의 mutation
    new_path = path[:]
    num_cities = len(path)

    # 첫 번째 도시와 마지막 도시를 제외한 나머지 도시에서 변이 수행
    for _ in range(int((num_cities - 2) * mutation_rate)):
        swap_idx1, swap_idx2 = random.sample(range(1, num_cities - 1), 2)  # 첫 번째 도시와 마지막 도시를 제외한 범위 내에서 랜덤 선택
        new_path[swap_idx1], new_path[swap_idx2] = new_path[swap_idx2], new_path[swap_idx1]

    return new_path

def genetic_algorithm(coordinates, max_iterations, num_expansions, mutation_rate,zero_genaration_num,zero_genaration_mutation_rate):
    root_path=farthest_neighbor(coordinates) # root 노드 패스 생성
    root_cost = calculate_cost(root_path, coordinates) # root 노드 패스의 cost 생성
    root = TreeNode(root_path, root_cost) # root 노드 생성

    # 0세대들 만들기 (nn ver)
    zero_generation_base_path=nearest_neighbor(coordinates)
    zero_generation_base_cost=calculate_cost(zero_generation_base_path,coordinates)
    zero_generation_base_node=TreeNode(zero_generation_base_path,zero_generation_base_cost) # 0 세대 base 노드
    expand_root_node(root,zero_generation_base_node,zero_genaration_num,coordinates,zero_genaration_mutation_rate)

    # 0세대들 만들기 (MST ver)
    MST_path,_,MST_cost=MST(coordinates)
    MST_node=TreeNode(MST_path,MST_cost) # 0 세대 base 노드
    expand_root_node(root,MST_node,zero_genaration_num,coordinates,zero_genaration_mutation_rate)

    # GA 돌리기
    current_iteration = 0
    while not stopping_condition(current_iteration, max_iterations):
        best_leaf_node = find_best_leaf_node(root)
        expand_tree(best_leaf_node, num_expansions, coordinates, mutation_rate)
        current_iteration += 1

    best_path = find_best_leaf_node(root).path
    best_cost = find_best_leaf_node(root).cost
    return best_cost, best_path

# 유전 알고리즘 끝 #

def value_iteration(distance_matrix, initial_policy, gamma=0.99, theta=1e-6):
    num_cities = len(distance_matrix)
    V = np.zeros(num_cities)
    policy = np.copy(initial_policy)

    while True:
        delta = 0
        count = 0
        for state in range(num_cities):
            v = V[state]
            next_values = [distance_matrix[state][next_state] + gamma * V[next_state] for next_state in range(num_cities) if next_state != state]
            min_value = min(next_values)
            V[state] = min_value
            policy[state] = np.argmin([distance_matrix[state][next_state] + gamma * V[next_state] for next_state in range(num_cities) if next_state != state])
            delta = max(delta, abs(v - V[state]))
            count = count+1
        if delta < theta:
            break

    return V, policy

# Extract Path and Cost
def extract_path(policy):
    path = [0]
    current_city = 0
    visited = set([0])

    while len(path) < len(policy):
        next_city = policy[current_city]
        if next_city in visited:
            break
        path.append(next_city)
        visited.add(next_city)
        current_city = next_city

    # Ensure all cities are visited by adding missing cities to the path
    missing_cities = set(range(len(policy))) - visited
    path.extend(missing_cities)

    path.append(0)  # Return to the starting city
    return path

# Main 함수
def main():
    # Parameters
    filepath = '../2024_AI_TSP.csv'
    max_iterations = 1000  # 세대수
    num_expansions = 1000  # 하나의 노드를 확장할때, 즉, 자식을 만들때 얼마나 많은 자식을 만들 것인지
    mutation_rate = 0.002  # GA를 돌릴때의 mutation rate
    zero_genaration_num = 10  # zero_genaration_num 만큼의 MST 기반 0세대 수
    zero_genaration_mutation_rate = 0.005  # 0세대를 만들때 base_node 기준으로 얼마나 mutation 할 것인지의 비율

    # Load data
    data = load_data(filepath)
    cities = data.cpu().numpy()
    distance_matrix = cdist(cities, cities, metric='euclidean')
    print(distance_matrix)
    # Run genetic algorithm
    best_cost, best_path = genetic_algorithm(data, max_iterations, num_expansions, mutation_rate, zero_genaration_num, zero_genaration_mutation_rate)
    #print("Initial 경로 : ", best_path)
    #print("Initial 비용 : ", best_cost)

    # Initial policy 생성
    initial_policy = np.array(best_path[:-1])

    # Value Iteration 수행
    V, optimized_policy = value_iteration(distance_matrix, initial_policy)

    # Extract the optimized path
    optimized_path = extract_path(optimized_policy)
    optimized_cost = sum(distance_matrix[optimized_path[i]][optimized_path[i+1]] for i in range(len(optimized_path)-1))
    
    # Print and plot the optimized path
    #plot_path(optimized_path, data)
    #print(f"Optimized 비용: {optimized_cost}")

if __name__ == "__main__":
    main()



[[0.         0.45273789 0.46327475 ... 0.26370405 0.88419354 0.66685916]
 [0.45273789 0.         0.35634079 ... 0.20310525 1.19281027 0.28660157]
 [0.46327475 0.35634079 0.         ... 0.28327294 1.34163155 0.32101102]
 ...
 [0.26370405 0.20310525 0.28327294 ... 0.         1.08767353 0.40699319]
 [0.88419354 1.19281027 1.34163155 ... 1.08767353 0.         1.47236525]
 [0.66685916 0.28660157 0.32101102 ... 0.40699319 1.47236525 0.        ]]


KeyboardInterrupt: 